# SVM Algorithm

## Classical imports and getting the training dataset

In [1]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Fitting models
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

Getting the clean datasets

In [2]:
#Getting the dataset
%run cleanDataV2.ipynb

0.38747862685781637


In [3]:
# Checking that the dataset is ok
df

,gender,washDishes,MrPropre,Antikal,Ariel,Dash,pods,powder,liquid,electricToothbrush,...,scent_High,scent_Medium,scent_Low,no_scent,magasin,moyenneSurface,superMarket,hyperMarket,drive,hardDiscount
0,1,1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,-1
1,1,1,1,1,1,-1,1,-1,-1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
2,1,1,-1,-1,1,1,-1,-1,1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,-1
3,1,-1,-1,-1,1,1,1,-1,-1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
4,1,-1,1,1,1,1,-1,-1,1,1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1,1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
11996,1,-1,-1,-1,-1,-1,-1,-1,1,1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
11997,1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
11998,-1,1,-1,-1,-1,-1,-1,-1,1,1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1


In [4]:
# A last clean to have float64 values
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

tf=clean_dataset(tf)

#Deleting the Id (we don't want our training to take into account the Id)
tf=tf.drop(columns='userId')

## SVM Model

In [5]:
#To see what the best parameters are
def best_model(model):
    print(model.best_score_)    
    print(model.best_params_)
    print(model.best_estimator_)

In [6]:
#splitting our dataset into one set of training and one set of test
y = tf.washDishes.values
x = tf.drop(['washDishes'], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [7]:
#SVM Regression
accuracies = {} 
param_grid = {'C': [50,100,150], 'gamma': [0.1,0.01,0.001],'probability':[True],'kernel': ['rbf']}
SVM_grid = GridSearchCV(SVC(), param_grid, cv=3, refit=True, verbose=0)
SVM_grid.fit(x_train, y_train)
acc = SVM_grid.score(x_test,y_test)*100
print(best_model(SVM_grid))
print("Test Accuracy {:.2f}%".format(acc))

## Prediction

In [9]:
#Predicting, and exporting to csv to make a submission
sol=pd.DataFrame({'userId':rf['userId']}) #creating the dataframe for the solution and getting the ID
rf=rf.drop(columns='userId') #dropping the id to have the same structure as the training dataset

y_pred = SVM_grid.fit(x_train, y_train).predict(rf) #the prediction
sol['washDishes'] = y_pred
sol["washDishes"] = sol["washDishes"].apply(lambda e: 'Auto' if e == 1 else 'Hand') #replacing the -1 and 1 with the original values
sol.to_csv('./DS_CentraleSupelec_ST42021/lrsolution.csv', index=False) #exporting to csv without the index
sol #checking the output

ValueError: X.shape[1] = 22 should be equal to 32, the number of features at training time